In [1]:
import pandas as pd

# Read Metadata
meta_df = pd.read_csv('/Users/brensikeg/Desktop/Final/archive/fer2013/fer2013.csv')

emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

meta_df = meta_df.rename(columns={"emotion":"label","pixels":"image"})
meta_df['emotion'] = meta_df['label'].apply(lambda x: emotions[int(x)])
meta_df = meta_df.drop(columns=['Usage'])

In [2]:
import numpy as np

meta_df["new_image"] = meta_df["image"].apply(lambda x: np.array(np.reshape(np.array(x.split(' ')).astype(int),(48,48))))
meta_df["svm"] = meta_df["image"].apply(lambda x: np.array(np.reshape(np.array(x.split(' ')).astype(int),(1,2304))))
meta_df = meta_df.drop(columns=['image'])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(meta_df["svm"]), np.array(meta_df["label"]), test_size=0.40, random_state=42)

In [8]:
X_train = np.concatenate(X_train, axis=0)
X_test = np.concatenate(X_test, axis=0)

In [9]:
from xgboost import XGBClassifier

xgbmodel = XGBClassifier(objective='multi:softprob', num_class= len(emotions), use_label_encoder=False)

print("fitting")
xgbmodel.fit(X_train, y_train)

print("scoring")
xgbmodel.score(X_test, y_test)

fitting
[17:36:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
scoring


0.4630442354580286

In [6]:
import dlib
import cv2
from imutils import face_utils

#step1: read the image
image = cv2.imread("/Users/brensikeg/Desktop/Final/test_image.jpeg")

#step2: converts to gray image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#step3: get HOG face detector and faces
hogFaceDetector = dlib.get_frontal_face_detector()
faces = hogFaceDetector(gray, 1)
predictor = dlib.shape_predictor("/Users/brensikeg/Desktop/Final/shape_predictor_68_face_landmarks.dat")

#step4: loop through each face and draw a rect around it
for (i, rect) in enumerate(faces):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    #draw a rectangle
    img = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    
    for (x, y) in shape:
        cv2.circle(image, (x, y), 2, (0, 0, 255), -1)
    
#step5: display the resulted image
cv2.imwrite('/Users/brensikeg/Desktop/Final/gottum.jpeg',img)
cv2.destroyAllWindows()